In [1]:
from samples.frontend import CacheConfig, in_out_nodes_generation, inst_block_gen, extract_mem_blocks
from samples.fixpoint import Fixpoint, FixpointType, fixpoint_nodes_generation
from pprint import pprint

In [2]:
""" All information needed for cache analysis. """

# Idents of all nodes.
all_nodes = ['Up', 'B_left', 'B_right', 'Down']
# All directed edges. The CFG is determined by nodes and edges.
edges = [('Up', 'B_left'), ('Up', 'B_right'), ('B_left', 'Down'), ('B_right', 'Down')]
# For instruction cache analysis, the instruction range of each basic block.
address_range = {'Up': (0x400784, 0x4007a4), 'B_left': (0x4007bc, 0x4007cb), 'B_right': (0x4007a8, 0x4007b8), 'Down': (0x4007cc, 0x4007d4)}

# Cache config.
cache_conf = CacheConfig(offset_len=6, set_index_len=8, assoc=4)

In [3]:
""" Get in_nodes and out_nodes of each basic block. """

in_nodes, out_nodes = in_out_nodes_generation(all_nodes, edges)

pprint(in_nodes)
pprint(out_nodes)

{'B_left': {'Up'},
 'B_right': {'Up'},
 'Down': {'B_right', 'B_left'},
 'Up': set()}
{'B_left': {'Down'},
 'B_right': {'Down'},
 'Down': set(),
 'Up': {'B_right', 'B_left'}}


In [4]:
""" Memory block of each basic block. """

mem_block_dict = {ident: inst_block_gen(*address_range[ident], cache_config=cache_conf) for ident in all_nodes}
pprint(mem_block_dict)

{'B_left': [<MemBlock tag:0x100 set_index:30>,
            <MemBlock tag:0x100 set_index:31>],
 'B_right': [<MemBlock tag:0x100 set_index:30>],
 'Down': [<MemBlock tag:0x100 set_index:31>],
 'Up': [<MemBlock tag:0x100 set_index:30>]}


In [5]:
""" Split memory block by set index. """

pprint(extract_mem_blocks(30, mem_block_dict))

{'B_left': [<MemBlock tag:0x100 set_index:30>],
 'B_right': [<MemBlock tag:0x100 set_index:30>],
 'Down': [],
 'Up': [<MemBlock tag:0x100 set_index:30>]}


In [6]:
""" Fixpoint nodes basic generation. """
""" Illustrated with must analysis, set_index=31 as example. """

fp_nodes = fixpoint_nodes_generation(nodes=all_nodes, in_nodes=in_nodes, out_nodes=out_nodes,
                                     cache_config=cache_conf, evicted=False,
                                     mem_block_dict=extract_mem_blocks(31, mem_block_dict))
pprint(fp_nodes)

[<Up ACC:[] IN:set(), OUT:{'B_right', 'B_left'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <B_left ACC:[256] IN:{'Up'}, OUT:{'Down'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <B_right ACC:[] IN:{'Up'}, OUT:{'Down'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <Down ACC:[256] IN:{'B_right', 'B_left'}, OUT:set() IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>]


In [7]:
fixpoint = Fixpoint(ty=FixpointType.Must, entry='Up', all_nodes=fp_nodes)
print(fixpoint.pse_topsort_seq)

fixpoint.run()
pprint(fp_nodes)

('Up', 'B_right', 'B_left', 'Down', 'Down')
[<Up ACC:[] IN:set(), OUT:{'B_right', 'B_left'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <B_left ACC:[256] IN:{'Up'}, OUT:{'Down'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{256}, {}, {}, {}]>,
 <B_right ACC:[] IN:{'Up'}, OUT:{'Down'} IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{}, {}, {}, {}]>,
 <Down ACC:[256] IN:{'B_right', 'B_left'}, OUT:set() IN_STAT:line:[{}, {}, {}, {}], OUT_STAT:line:[{256}, {}, {}, {}]>]
